### Agents
Agents in essence are LLMs that accomplish a specific task. They can be supplemented with tools, structured output, and handoff to other agents.

In [12]:
%pip install openai-agents python-dotenv -qU

Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.


In [14]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads from .env file

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")


In [16]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="You are a helpful assistant. Respond on in all caps.",
    model="gpt-4o-mini"
)

result = await Runner.run(agent, "Hello! What's up dude")
result.final_output

'HELLO! NOT MUCH, JUST HERE TO HELP YOU. HOW ABOUT YOU?'

In [17]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller for seniors, today's their scribble day. You are given a name and you need to tell a joke about it.",
)

topic = "Pragun"
result = await Runner.run(joke_agent, topic)
result.final_output

'Why did Pragun bring a pencil to the party?\n\nBecause he heard it was going to be a "write" good time!'

In [18]:
language_agent = Agent(
    name="Language Agent",
    instructions="You are a language expert. You are given a joke and you need to rewrite it in Hindi.",
)

joke_result = await Runner.run(joke_agent, topic)
translated_result = await Runner.run(language_agent, f"Translate this joke to Hindi: {joke_result.final_output}")
print(f"Original joke:\n{joke_result.final_output}\n")
print(f"Translated joke:\n{translated_result.final_output}")

Original joke:
Why did Pragun bring a pencil to the senior scribble day?

Because he heard it was the "write" thing to do!

Translated joke:
प्रगुण सीनियर स्क्रिबल डे पर पेंसिल क्यों लाया?

क्योंकि उसने सुना कि यही सही काम है!


### Structured Outputs
Structured outputs are a way to format the output of an LLM in a structured manner. This can be useful for tasks that require specific formatting or data extraction.

In [24]:
from pydantic import BaseModel
from agents import Agent, Runner

class Recipe(BaseModel):
    title: str
    ingredients: list[str]
    cooking_time: int # in minutes
    servings: int

recipe_agent = Agent(
    name="Recipe Agent", 
    instructions=("You are an agent for creating recipes. You will be given the name of a food and your job"
                  " is to output that as an actual detailed recipe. The cooking time should be in minutes."),
    output_type=Recipe
)

response = await Runner.run(recipe_agent, "aloo ka paratha")
recipe = response.final_output

print(recipe)


title='Aloo Ka Paratha' ingredients=['2 cups whole wheat flour', '1 cup water (for kneading)', '1 teaspoon salt', '3 medium-sized potatoes, boiled and mashed', '1 teaspoon cumin seeds', '1 teaspoon red chili powder', '1 teaspoon garam masala', '1 tablespoon finely chopped coriander leaves', '1 green chili, finely chopped', '2 tablespoons oil or ghee', 'Salt to taste', 'Butter or ghee for cooking'] cooking_time=45 servings=4


### Tool Calling
Tool calling is a way to extend the capabilities of an LLM by allowing it to call external tools or APIs. This can be useful for tasks that require access to external data or services.

In [20]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
    print(f"Getting weather for {city}")
    return "sunny"

@function_tool
def get_temperature(city: str) -> str:
    print(f"Getting temperature for {city}")
    return "70 degrees"

agent = Agent(
    name="Weather Agent",
    instructions="You are the local weather agent. You are given a city and you need to tell the weather and temperature. For any unrelated queries, say I cant help with that.",
    tools=[get_weather, get_temperature]
)

result = await Runner.run(agent, "Dallas")
result.final_output

Getting weather for Dallas
Getting temperature for Dallas


'The weather in Dallas is currently sunny, with a temperature of 70 degrees.'

In [21]:
from agents import WebSearchTool

news_agent = Agent(
    name="News Reporter",
    instructions="You are a news reporter. Your job is to find recent news articles on the internet about US politics.",
    tools=[WebSearchTool()]
)

result = await Runner.run(news_agent, "find news")
print(result.final_output)

Here are some recent developments in U.S. politics:

**Democratic Party's Internal Challenges**

The Democratic Party is facing internal divisions, particularly highlighted during recent budget battles. Some members supported Republican bills, while others opposed them, complicating the party's unified stance against the Trump administration. Calls for new leadership, including critiques of figures like Chuck Schumer, have emerged. The party is urged to establish a clear message and engage voters actively ahead of the midterm elections. ([ft.com](https://www.ft.com/content/13b4c22d-23e0-4f62-bd5f-5a7afd60e83d?utm_source=openai))

**Donald Trump's Influence on Cultural Institutions**

Former President Donald Trump continues to assert influence over American institutions. As chair of the John F. Kennedy Center for the Performing Arts, he aims to remove "woke" influences and discourage performances he deems "anti-American." This cultural campaign parallels actions in academia, particularl